In [1]:
import cv2
import numpy as np

class Land():
    def __init__(self):
        n = 15
        land_0 = np.zeros((n,n))
        for _ in range(20):
            land_0[np.random.randint(2,n-2), np.random.randint(2,n-2)] = 100.
        
        for i in range(4):
            land_1 = cv2.pyrUp(land_0)
            land_0 = cv2.blur(land_1,(5,5))
        
        land = land_0.astype(np.dtype('uint8'))
        
        self.growth = land
        self.land = land.copy()
    
    def grow(self):
        self.land += self.growth
        

In [21]:
import numpy as np

class Bot():
    def __init__(self,xy,th,fat=10,W=None):
        self.xy = np.array(xy)
        self.th = th
        self.fat = fat
        if not W: self.W = np.random.rand(6)-0.5
        else: self.W = W
    
    def feel(self, land):
        xl,yl = self.xy + 2 * np.array([np.cos(self.th-1),  
                                        np.sin(self.th+1)])
        xr,yr = self.xy + 2 * np.array([np.cos(self.th-1), 
                                        np.sin(self.th-1)])
        return land.land[int(xl),int(yl)], land.land[int(xr),int(yr)]
    
    def move(self,land):
        Fl,Fr = self.feel(land)
        Vl = sigmoid(self.W[0] + self.W[1]*Fl + self.W[2]*Fr )
        Vr = sigmoid(self.W[3] + self.W[4]*Fl + self.W[5]*Fr )
        if Vl == Vr: 
            dth = 0
        else:
            r = (Vl+Vr)/(Vl-Vr)
            dth = -Vl/(r+0.5)
        print 'Vl=%.2g, Vr=%.2g \t dth=%.2g' %(Vl,Vr,dth)
        self.th += dth
        self.xy += (Vl+Vr) / 2. * np.array([np.cos(self.th), 
                                           np.sin(self.th)])
    def in_land(self,land):
        W,H = np.shape(land.land)
        if self.xy[0]<0 or int(self.xy[0])>=W-1: return False
        if self.xy[1]<0 or int(self.xy[1])>=H-1: return False
        return True
    
    def eat(self,land):
        pass
    
    def reproduce(self):
        if self.fat > 99:
            self.fat = self.fat/3
            fat = self.fat/3
            xy = self.xy
            th = self.th + np.pi
            return Bot(xy=xy,th=th,fat=fat)
        else:
            return None
    
    def __repr__(self):
        return '< bot xy=%s, th=%.2g, fat=%i >\n' % (self.xy, self.th, self.fat)

def sigmoid(x): return 1./(1.+np.exp(-x))

In [22]:
land = Land()
W,H = np.shape(land.land)
Bots = [ Bot(xy=(np.random.uniform(0,W),np.random.uniform(0,H)),
             th=np.random.uniform(-np.pi,np.pi)) for i in range(5)]

for i in range(100):
    new_bots = []
    for bot in Bots:
        bot.move(land)
        if bot.in_land(land):
            bot.eat(land)
            new = bot.reproduce()
            if new: new_bots.append(new)
            new_bots.append(bot)
    print '%i Population: %i' %(i,len(new_bots))
    Bots = new_bots[:]
    
    

Vl=0.43, Vr=0.39 	 dth=-0.021
Vl=1, Vr=2.6e-05 	 dth=-0.67
Vl=0.65, Vr=0.99 	 dth=0.15
Vl=0.97, Vr=0.017 	 dth=-0.63
Vl=0.89, Vr=0.39 	 dth=-0.29
0 Population: 5
Vl=0.43, Vr=0.39 	 dth=-0.021
Vl=1, Vr=2e-05 	 dth=-0.67
Vl=0.65, Vr=0.99 	 dth=0.15
Vl=0.96, Vr=0.017 	 dth=-0.62
Vl=0.89, Vr=0.39 	 dth=-0.29
1 Population: 5
Vl=0.43, Vr=0.39 	 dth=-0.021
Vl=1, Vr=2e-05 	 dth=-0.67
Vl=0.65, Vr=0.99 	 dth=0.15
Vl=0.97, Vr=0.017 	 dth=-0.63
Vl=0.88, Vr=0.4 	 dth=-0.28
2 Population: 5
Vl=0.43, Vr=0.39 	 dth=-0.021
Vl=1, Vr=2e-05 	 dth=-0.67
Vl=0.65, Vr=0.99 	 dth=0.15
Vl=0.98, Vr=0.025 	 dth=-0.63
Vl=0.88, Vr=0.4 	 dth=-0.28
3 Population: 5
Vl=0.43, Vr=0.39 	 dth=-0.021
Vl=1, Vr=2e-05 	 dth=-0.67
Vl=0.63, Vr=0.98 	 dth=0.16
Vl=0.98, Vr=0.025 	 dth=-0.63
Vl=0.88, Vr=0.4 	 dth=-0.28
4 Population: 4
Vl=1, Vr=2.6e-05 	 dth=-0.67
Vl=0.63, Vr=0.98 	 dth=0.16
Vl=0.97, Vr=0.017 	 dth=-0.63
Vl=0.88, Vr=0.4 	 dth=-0.28
5 Population: 4
Vl=1, Vr=2.6e-05 	 dth=-0.67
Vl=0.63, Vr=0.98 	 dth=0.16
Vl=0.97, Vr=0

In [288]:
print 'feels:', bot.feel(land)
bot.move(land)
print bot.xy

feels: (1, 0)
Vl,Vr: 0.528810997213 0.723845310322 	dth: 0.0892847320759
[ 49.87730628  84.29629857]
